## Librairies

In [1]:
import warnings
warnings.filterwarnings("ignore")

import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

import xscale.spectral.fft as xfft
from xscale.spectral.tools import plot_spectrum
import xscale.signal.generator as xgen


import matplotlib.pyplot as plt
params = {'figure.figsize' : (18, 10),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)


## Dataset

### Format netcdf

In [ ]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/'
tfilename = sorted(glob.glob(data_dir+'eNATL60-BLBT02*-S/*/eNATL60-BLBT02*_1h_*_gridT-2D_*.nc'))
filename =tfilename[0]


In [ ]:
%time dsn=xr.open_mfdataset(tfilename)['sossheig']


In [ ]:
dsn

In [ ]:
print(dsn.nbytes/1e9)

### Format zarr

In [2]:
%time ds=xr.open_zarr('/store/albert7a/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')
ds_sorted=ds.sortby('time_counter')

CPU times: user 157 ms, sys: 83.1 ms, total: 241 ms
Wall time: 1.61 s


In [ ]:
ds

In [ ]:
print(ds.nbytes/1e9)

In [3]:
ds_sorted=ds.sortby('time_counter')

### Selecting time frame and 1 point

In [4]:
ssh_FMA=ds_sorted.sel(time_counter=slice('2010-02-01','2010-04-30'))['sossheig']
ssh_FMA_1pt=ssh_FMA[:,2000,4000]
ssh_ASO=ds_sorted.sel(time_counter=slice('2009-08-01','2009-10-31'))['sossheig']
ssh_ASO_1pt=ssh_ASO[:,2000,4000]

### Xarray plotting capabilities

In [ ]:
ssh_FMA[12].plot()
plt.scatter(4000,2000,marker='x')

In [ ]:
ssh_FMA_1pt.plot()

In [ ]:
ssh_FMA_1pt.plot(label='full data')
ssh_FMA_1pt.rolling(time_counter=24,center=True).mean().plot(label='rolling daily mean')

## FFT decomposition and power spectral density computation

In [ ]:
NPADDING = 2
SPtime_FMA = xfft.fft(ssh_FMA_1pt, dim='time_counter', dx=1., detrend='mean',nfft=ssh_FMA_1pt.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA).load()
freqs_FMA=MEANPSD_FMA.f_time_counter.values
freqs_FMA = freqs_FMA[NPADDING::] 

SPtime_ASO = xfft.fft(ssh_ASO_1pt, dim='time_counter', dx=1., detrend='mean',nfft=ssh_ASO_1pt.shape[0]*NPADDING,tapering=True)
MEANPSD_ASO=xfft.psd(SPtime_ASO).load()
freqs_ASO=MEANPSD_ASO.f_time_counter.values
freqs_ASO = freqs_ASO[NPADDING::] 


In [ ]:
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)

ax.loglog(freqs_FMA,MEANPSD_FMA[NPADDING::],'k',label='winter')
ax.loglog(freqs_ASO,MEANPSD_ASO[NPADDING::],'r',label='summer')
ax.set_xlim(1e-3,1)
ax.grid(which='both',axis='both')
plt.xlabel('Frequency (cpd)')
plt.ylabel('PSD (m2/cpd)')
plt.legend()
plt.title('Temporal Spectrum of SSH at one point')

## Same computation for 100x100 points

### Computational ressources

In [9]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='demo',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(196)
cluster

In [10]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://172.30.100.1:32785 Dashboard: http://172.30.100.1:44174/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [11]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7591846      allc     demo albert7a PD       0:00      1 (Priority)
           7591847      allc     demo albert7a PD       0:00      1 (Priority)
           7591844      allc     demo albert7a PD       0:00      1 (Priority)
           7591845      allc     demo albert7a PD       0:00      1 (Priority)
           7591843      allc     demo albert7a PD       0:00      1 (Priority)
           7591841      allc     demo albert7a PD       0:00      1 (Priority)
           7591842      allc     demo albert7a PD       0:00      1 (Priority)
           7591151       all    zsrst albert7a PD       0:00      1 (Resources)
           7591154       all zmergxio albert7a PD       0:00      1 (Resources)
           7591182       all    zsrst albert7a PD       0:00      1 (Resources)
           7591184       all zmergxio albert7a PD       0:00      1 (Resources)
           7591211       all    zsrst albe

### Data selection

In [ ]:
ssh_FMA_reg=ssh_FMA[:,2000:2100,4000:4100]
ssh_ASO_reg=ssh_ASO[:,2000:2100,4000:4100]
ssh_FMA[12].plot()
plt.plot([4000, 4100], [2000, 2000])
plt.plot([4000, 4100], [2100, 2100])
plt.plot([4000, 4000], [2000, 2100])
plt.plot([4100, 4100], [2000, 2100])


In [ ]:
NPADDING = 2
SPtime_FMA = xfft.fft(ssh_FMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=ssh_FMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA).mean(dim='x').mean(dim='y').load()
freqs_FMA=MEANPSD_FMA.f_time_counter.values
freqs_FMA = freqs_FMA[NPADDING::] 

SPtime_ASO = xfft.fft(ssh_ASO_reg, dim='time_counter', dx=1., detrend='mean',nfft=ssh_ASO_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_ASO=xfft.psd(SPtime_ASO).mean(dim='x').mean(dim='y').load()
freqs_ASO=MEANPSD_ASO.f_time_counter.values
freqs_ASO = freqs_ASO[NPADDING::] 


In [ ]:
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)

ax.loglog(freqs_FMA,MEANPSD_FMA[NPADDING::],'k',label='winter')
ax.loglog(freqs_ASO,MEANPSD_ASO[NPADDING::],'r',label='summer')
ax.set_xlim(1e-3,1)
ax.grid(which='both',axis='both')
plt.xlabel('Frequency (cpd)')
plt.ylabel('PSD (m2/cpd)')
plt.legend()
plt.title('Temporal Spectrum of SSH at one point')